# Google Colab: Grabar un dataset
Ejemplo para grabar un dataset de clasificación de imágenes usando la webcam.

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import os
import glob
from pathlib import Path

## Funciones auxiliares
Conversión de formato de imagen y búsqueda del último id de imagen.

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

def get_last_counter(images_path):
	files_list = glob.glob(os.path.join(images_path, '*.png'))
	return len(files_list)

## Capturar imágenes
Usamos código en javascript para manejar la cámara desde el navegador y, al pulsar el botón "Capturar" se almacena una imagen. Hay que configurar el nombre de la clase que se está capturando y cada pulsación de botón genera una imagen dentro de la carpeta correspondiente a dicha clase.

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capturar';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # get photo data
  data = eval_js('takePhoto({})'.format(quality))

  # get OpenCV format image
  img = js_to_image(data)
  # save image
  cv2.imwrite(filename, img)

In [ ]:
####### Cambiar estas variables #######
outdir = './dataset'
class_name = 'perro'
Path(os.path.join(outdir, class_name)).mkdir(parents=True, exist_ok=True)

In [ ]:
try:
  record_data = True
  counter = get_last_counter(os.path.join(outdir, class_name))
  while (record_data):
    file_path = os.path.join(outdir, class_name, "{:09d}.png".format(counter))
    take_photo(file_path)
    print(file_path)
    counter = counter + 1
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
! tar -czf dataset.tar.gz dataset